In [369]:
#Yiwei Yu
#5708729817

In [372]:
import pandas as pd
import numpy as np
import random
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import hamming_loss
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import random
from sklearn.metrics import silhouette_score
from sklearn.svm import LinearSVC
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

In [337]:
random.seed(222)

<h2>1.Multi-class and Multi-Label Classification Using Support Vector Machines</h2>

<h3>(a) Choose 70% of the data randomly as the training set.</h3>

In [342]:
data = pd.read_csv("../data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv")
all_ind = list(data.index)
train_num = np.floor(len(all_ind)*0.7)
train_ind = random.sample(list(all_ind),int(train_num))
test_ind = []
for i in all_ind:
    if i not in train_ind:
        test_ind.append(i)

In [343]:
train_data = data.iloc[train_ind,:].reset_index().drop('index',axis = 1)
train_data

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.222841,0.124870,0.538781,0.198879,0.062832,-0.136344,0.013634,0.267089,0.076896,...,0.182757,0.082640,-0.061288,-0.177844,-0.015137,0.232737,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,15
1,1.0,0.279315,0.321542,0.645042,0.238846,0.041848,-0.137007,0.071154,0.260744,0.000629,...,0.216176,-0.101481,-0.163043,-0.062751,0.159484,0.183786,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,24
2,1.0,0.406430,0.229384,0.551461,0.227854,-0.019469,-0.091908,0.039312,0.323062,0.024180,...,0.264568,0.029546,-0.128165,-0.120402,0.015387,0.208457,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,22
3,1.0,0.517017,0.426751,0.197185,0.020329,0.166751,0.047792,-0.028725,0.094501,0.079356,...,-0.000104,-0.010787,0.023791,-0.003139,-0.004702,0.045808,Hylidae,Hypsiboas,HypsiboasCordobae,40
4,1.0,0.146944,-0.009583,0.352534,0.313435,0.197599,0.015352,-0.216492,-0.133865,0.278309,...,-0.043595,-0.052536,-0.024106,0.016081,0.062506,0.042285,Leptodactylidae,Adenomera,AdenomeraAndre,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5031,1.0,0.099985,-0.048752,0.420582,0.211123,0.069708,-0.065316,0.074026,0.327846,0.137529,...,0.272311,0.158587,-0.069248,-0.191575,-0.002584,0.237368,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,21
5032,1.0,0.847515,0.882058,0.394028,-0.072178,0.353863,0.138748,0.144420,0.282237,-0.207722,...,-0.191531,-0.033739,0.092026,0.148543,0.099929,-0.111212,Hylidae,Hypsiboas,HypsiboasCinerascens,37
5033,1.0,0.062590,0.130510,0.460425,0.085099,-0.033405,-0.112546,0.052770,0.291180,0.074378,...,0.176763,-0.016809,-0.095981,-0.101811,0.101536,0.165841,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,18
5034,1.0,0.167250,0.271196,0.310860,0.164596,0.176131,0.077902,-0.136789,-0.032877,0.109147,...,0.093964,0.034244,0.012648,-0.020759,-0.050157,0.013067,Hylidae,Hypsiboas,HypsiboasCordobae,43


In [344]:
test_data = data.iloc[test_ind,:].reset_index().drop('index',axis = 1)
test_data

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.145130,-0.033660,0.284166,0.279537,0.175211,0.005791,-0.183329,-0.158483,0.192567,...,-0.055978,-0.048219,-0.056637,-0.022419,0.070085,0.021419,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.271326,0.027777,0.375738,0.385432,0.272457,0.098192,-0.173730,-0.157857,0.207181,...,-0.120723,-0.112607,-0.156933,-0.118527,-0.002471,0.002304,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.148539,-0.096910,0.257523,0.260881,0.312603,0.134134,-0.216262,-0.189334,0.261960,...,-0.034082,-0.120716,-0.100800,-0.001992,0.111462,0.103637,Leptodactylidae,Adenomera,AdenomeraAndre,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154,1.0,-0.512794,0.056322,0.259677,0.030140,0.369783,-0.117154,-0.189292,0.248948,0.193566,...,0.096589,0.255261,0.047039,-0.016643,-0.046598,-0.055862,Hylidae,Scinax,ScinaxRuber,60
2155,1.0,-0.507564,-0.249969,0.031781,-0.079888,0.484274,0.125143,-0.069555,0.114265,0.099647,...,0.011524,0.111998,0.015886,0.008636,-0.021933,-0.069692,Hylidae,Scinax,ScinaxRuber,60
2156,1.0,-0.512599,-0.171956,0.325813,0.169600,0.421567,-0.123749,-0.298284,0.089382,0.243902,...,0.021225,0.157321,0.042847,0.006852,0.005439,-0.013693,Hylidae,Scinax,ScinaxRuber,60
2157,1.0,-0.554504,-0.337717,0.035533,0.034511,0.443451,0.093889,-0.100753,0.037087,0.081075,...,0.069430,0.071001,0.021591,0.052449,-0.021860,-0.079860,Hylidae,Scinax,ScinaxRuber,60


<h3>b.i Research exact match and hamming score/ loss methods for evaluating multi-label classification and use them in evaluating the classifiers in this problem.</h3>

<p>Exact Match:The percentage of samples that have classified all labels correctly. The drawback of this metric is that for multi-class classfication, it ignores all the partially correct matches. </p>
<p>Hamming Loss: The proportion of incorrectly predicted labels over the total number of labels. It takes the partially correct matches into account</p>
<p>Hamming Score: 1- Hamming Loss </p>

In [ ]:
def metric_compute(true_tag,predict_tag):
    mismatch = np.count_nonzero(np.not_equal(true_tag,predict_tag))
    alltag = true_tag.shape[0] * true_tag.shape[1]
    print('hamming loss:')
    print(mismatch / alltag)
    
    new_true = []
    new_pre = []
    for i in range(0,true_tag.shape[1]):
        new_true.append([true_tag[0][i],true_tag[1][i]])
        new_pre.append([predict_tag[0][i],predict_tag[1][i]])
    count_diff = 0
    for i in range(0,len(new_true)):
        if new_true[i] == new_pre[i]:
            count_diff += 1
    print('exact match:')
    print(count_diff / len(new_true))
    return (mismatch / alltag, count_diff / len(new_true) )

<h3>b.ii Train a SVM for each of the labels, using Gaussian kernels and one versus
all classifiers</h3>

<p>Raw Data</p>

In [345]:
train_d = train_data.iloc[:,:-4]
train_y_f = train_data[['Family']].values.ravel()
train_y_g = train_data[['Genus']].values.ravel()
train_y_s = train_data[['Species']].values.ravel()

test_d = test_data.iloc[:,:-4]
test_y_f = test_data[['Family']].values.ravel()
test_y_g = test_data[['Genus']].values.ravel()
test_y_s = test_data[['Species']].values.ravel()

In [346]:
svc = svm.SVC(kernel = 'rbf')
model_family = GridSearchCV(svc,{'C':[10 ** i for i in range(-3,7)],'gamma':[0.5,1.0,1.5,2.0]},cv = 10)
model_family.fit(train_d, np.array(train_y_f))
print("Best CV Score(Family):")
print(model_family.best_score_)
print("Best Parameter(Family):")
print(model_family.best_params_)

Best CV Score(Family):
0.9936460601470541
Best Parameter(Family):
{'C': 100, 'gamma': 2.0}


In [347]:
model_genus = GridSearchCV(svc,{'C':[10 ** i for i in range(-3,7)],'gamma':[0.5,1.0,1.5,2.0]},cv = 10)
model_genus.fit(train_d, np.array(train_y_g))
print("Best CV Score(Genus):")
print(model_genus.best_score_)
print("Best Parameter(Genus):")
print(model_genus.best_params_)

Best CV Score(Genus):
0.9908674934519863
Best Parameter(Genus):
{'C': 10, 'gamma': 1.0}


In [348]:
model_species = GridSearchCV(svc,{'C':[10 ** i for i in range(-3,7)],'gamma':[0.5,1.0,1.5,2.0]},cv = 10)
model_species.fit(train_d, np.array(train_y_s))
print("Best CV Score(Species):")
print(model_species.best_score_)
print("Best Parameter(Species):")
print(model_species.best_params_)

Best CV Score(Species):
0.991263135472877
Best Parameter(Species):
{'C': 10, 'gamma': 2.0}


In [349]:
plabel_family = model_family.predict(test_d)
plabel_genus = model_genus.predict(test_d)
plabel_spe = model_species.predict(test_d)

all_pre = np.array([plabel_family,plabel_genus,plabel_spe])
all_true = np.array([test_y_f,test_y_g,test_y_s])
print('Final Result with optimal Parameters: ')
metric_compute(all_true,all_pre)

Final Result with optimal Parameters: 
hamming loss:
0.008028408213679172
exact match:
0.9898100972672533


(0.008028408213679172, 0.9898100972672533)

<p>Standardized</p>

In [351]:
my_scaler = StandardScaler()
train_standard = my_scaler.fit_transform(train_d)
test_standard = my_scaler.fit_transform(test_d)

<h3>b.iii Repeat 1(b)ii with L1-penalized SVMs.</h3>

In [352]:
svc = LinearSVC(penalty = 'l1',dual = False,max_iter = 3000)
model_family = GridSearchCV(svc,{'C':[10 ** i for i in range(-3,7)]},cv = 10)
model_family.fit(train_standard, np.array(train_y_f))
print("Best CV Score(Family-L1):")
print(model_family.best_score_)
print("Best Parameter(Family-L1):")
print(model_family.best_params_)

Best CV Score(Family-L1):
0.9368605825365268
Best Parameter(Family-L1):
{'C': 1}


In [353]:
svc = LinearSVC(penalty = 'l1',dual = False,max_iter = 6000)
model_genus = GridSearchCV(svc,{'C':[10 ** i for i in range(-3,7)]},cv = 10)
model_genus.fit(train_standard, np.array(train_y_g))
print("Best CV Score(Genus-L1):")
print(model_genus.best_score_)
print("Best Parameter(Genus-L1):")
print(model_genus.best_params_)

Best CV Score(Genus-L1):
0.9569144655874279
Best Parameter(Genus-L1):
{'C': 100}


In [354]:
svc = LinearSVC(penalty = 'l1',dual = False,max_iter = 6000)
model_species = GridSearchCV(svc,{'C':[10 ** i for i in range(-3,7)]},cv = 10)
model_species.fit(train_standard, np.array(train_y_s))
print("Best CV Score(Species-L1):")
print(model_species.best_score_)
print("Best Parameter(Species-L1):")
print(model_species.best_params_)

Best CV Score(Species-L1):
0.9622747641137304
Best Parameter(Species-L1):
{'C': 1}


In [355]:
plabel_family = model_family.predict(test_standard)
plabel_genus = model_genus.predict(test_standard)
plabel_spe = model_species.predict(test_standard)

all_pre = np.array([plabel_family,plabel_genus,plabel_spe])
all_true = np.array([test_y_f,test_y_g,test_y_s])
print('Final Result with optimal Parameters(L1-Penalty): ')
metric_compute(all_true,all_pre)

Final Result with optimal Parameters(L1-Penalty): 
hamming loss:
0.05697081982399259
exact match:
0.9166280685502547


(0.05697081982399259, 0.9166280685502547)

<h3>b.iv Repeat 1(b)iii by using SMOTE or any other method you know to remedy class imbalance.</h3>

In [356]:
Counter(train_y_f)

Counter({'Leptodactylidae': 3076,
         'Hylidae': 1515,
         'Dendrobatidae': 396,
         'Bufonidae': 49})

In [357]:
oversample = SMOTE()
train_d_f,tag_f_smote = SMOTE().fit_resample(train_standard,train_y_f)
train_d_g,tag_g_smote = SMOTE().fit_resample(train_standard,train_y_g)
train_d_s,tag_s_smote = SMOTE().fit_resample(train_standard,train_y_s)

In [360]:
print(Counter(tag_f_smote))

Counter({'Leptodactylidae': 3076, 'Hylidae': 3076, 'Dendrobatidae': 3076, 'Bufonidae': 3076})


<h5>Best CV Score and Parameters (Family-L1-SMOTE)</h5>

In [361]:
svc = LinearSVC(penalty = 'l1',dual = False,max_iter = 3000)
model_family = GridSearchCV(svc,{'C':[10 ** i for i in range(-3,7)]},cv = 10)
model_family.fit(train_d_f, np.array(tag_f_smote))
print(model_family.best_score_)
print(model_family.best_params_)

0.9454655808946393
{'C': 1000}


<h5>Best CV Score and Parameters (Genus-L1-SMOTE)</h5>

In [362]:
svc = LinearSVC(penalty = 'l1',dual = False,max_iter = 10000)
model_genus = GridSearchCV(svc,{'C':[10 ** i for i in range(-3,7)]},cv = 10)
model_genus.fit(train_d_g, np.array(tag_g_smote))
print(model_genus.best_score_)
print(model_genus.best_params_)

0.9561599279311542
{'C': 1000}


<h5>Best CV Score and Parameters (Species-L1-SMOTE)</h5>

In [373]:
svc = LinearSVC(penalty = 'l1',dual = False,max_iter = 10000)
model_species = GridSearchCV(svc,{'C':[10 ** i for i in range(-3,7)]},cv = 10)
model_species.fit(train_d_s, np.array(tag_s_smote))
print(model_species.best_score_)
print(model_species.best_params_)

0.9578881987577639
{'C': 10}


In [376]:
plabel_family = model_family.predict(test_standard)
plabel_genus = model_genus.predict(test_standard)
plabel_spe = model_species.predict(test_standard)

all_pre = np.array([plabel_family,plabel_genus,plabel_spe])
all_true = np.array([test_y_f,test_y_g,test_y_s])
print('Final Result with optimal Parameters(L1-Penalty-SMOTE): ')
metric_compute(all_true,all_pre)

Final Result with optimal Parameters(L1-Penalty-SMOTE): 
hamming loss:
0.07441716844218002
exact match:
0.8679944418712366


(0.07441716844218002, 0.8679944418712366)

<h3>Report your conclusions about the classifiers you trained:</h3>
<h5> 1.Gaussian Kernel with raw feature:</h5>
<p>Hamming Loss:0.0080</p>
<p>Exact Match:0.9898</p>
<h5> 2.L1-penalized SVM:</h5>
<p>Hamming Loss:0.05697</p>
<p>Exact Match:0.9166</p>
<h5> 2.L1-penalized SVM with SMOTE:</h5>
<p>Hamming Loss:0.0744</p>
<p>Exact Match:0.86799</p>
<p><b>Conclusion: </b>Based upon the hamming loss and exact match ratio, the SVM model with Gausian Kernel performs the best with the lowest hamming loss and highest exact match ratio</p>

<h2>2.K-Means Clustering on a Multi-Class and Multi-Label Data Set.</h2>

<p>Monte-Carlo Simulation:
Perform the following procedures 50 times, and report
the average and standard deviation of the 50 Hamming Distances that you calculate</p>

In [365]:
data_x = data.iloc[:,:-4]
data_y = data.iloc[:,-4:-1]


In [366]:
def monte_carlo(seed_num):
    max_score = -999
    best_k = 1
    for k_num in range(2,51):
        model_kmeans = KMeans(n_clusters = k_num,random_state = seed_num).fit(data_x)
    #predict_now = model_kmeans.predict(data_x)
        predict_label = model_kmeans.labels_
        score_now = silhouette_score(data_x,predict_label)
        if score_now > max_score:
            best_k = k_num
            max_score = score_now
    print('Best K for this time is : ' + str(best_k))
    model_kmeans = KMeans(n_clusters = best_k,random_state = seed_num).fit(data_x)
    predict_label = model_kmeans.labels_
    
    predict_result = data_y.copy()
    predict_result
    for i in range(0,best_k):
        label_ind = np.where(predict_label == i)[0]
        major_label = data_y.loc[label_ind].groupby(['Family','Genus','Species']).size().sort_values(ascending = False).index[0]
        for i in label_ind:
            predict_result.loc[i]  = major_label
    ##compute hamming loss
    mismatch = np.count_nonzero(np.not_equal(data_y,predict_result))
    alltag = data_y.shape[0] * data_y.shape[1]
    hamming_loss = mismatch/alltag
    ##compute hamming score
    hamming_score = 1 - hamming_loss
    ##compute hamming distance
    hamming_distance = mismatch / data_y.shape[0]
    return (hamming_loss,hamming_score,hamming_distance)


In [368]:
list_loss = []
list_score = []
list_dist = []
for i in range(0,50):
    print('Time:'+str(i) + " ")
    hloss,hscore,hdist = monte_carlo(i+22)
    print('Hamming Distance: ' + str(hdist))
    print('Hamming Score: ' + str(hscore))
    print('Hamming Loss: ' + str(hloss))
    list_loss.append(hloss)
    list_score.append(hscore)
    list_dist.append(hdist)
    print('--------------------------------------------------------------------------')
    
    

Time:0 
Best K for this time is : 4
Hamming Distance: 0.66726893676164
Hamming Score: 0.7775770210794533
Hamming Loss: 0.2224229789205467
--------------------------------------------------------------------------
Time:1 
Best K for this time is : 4
Hamming Distance: 0.6674079221681724
Hamming Score: 0.7775306926106093
Hamming Loss: 0.22246930738939077
--------------------------------------------------------------------------
Time:2 
Best K for this time is : 4
Hamming Distance: 0.66726893676164
Hamming Score: 0.7775770210794533
Hamming Loss: 0.2224229789205467
--------------------------------------------------------------------------
Time:3 
Best K for this time is : 4
Hamming Distance: 0.66726893676164
Hamming Score: 0.7775770210794533
Hamming Loss: 0.2224229789205467
--------------------------------------------------------------------------
Time:4 
Best K for this time is : 4
Hamming Distance: 0.66726893676164
Hamming Score: 0.7775770210794533
Hamming Loss: 0.2224229789205467
-------

Best K for this time is : 4
Hamming Distance: 0.66726893676164
Hamming Score: 0.7775770210794533
Hamming Loss: 0.2224229789205467
--------------------------------------------------------------------------
Time:40 
Best K for this time is : 4
Hamming Distance: 0.66726893676164
Hamming Score: 0.7775770210794533
Hamming Loss: 0.2224229789205467
--------------------------------------------------------------------------
Time:41 
Best K for this time is : 4
Hamming Distance: 0.735371785962474
Hamming Score: 0.754876071345842
Hamming Loss: 0.24512392865415797
--------------------------------------------------------------------------
Time:42 
Best K for this time is : 4
Hamming Distance: 0.6653231410701876
Hamming Score: 0.7782256196432707
Hamming Loss: 0.22177438035672922
--------------------------------------------------------------------------
Time:43 
Best K for this time is : 4
Hamming Distance: 0.66726893676164
Hamming Score: 0.7775770210794533
Hamming Loss: 0.2224229789205467
----------

In [370]:
print("Mean(Hamming Loss): " + str(np.mean(list_loss)))
print("Std.(Hamming Loss): " + str(np.std(list_loss)))
print(" ")
print("Mean(Hamming Score): " + str(np.mean(list_score)))
print("Std.(Hamming Score): " + str(np.std(list_score)))
print(" ")
print("Mean(Hamming Distance): " + str(np.mean(list_dist)))
print("Std.(Hamming Distance): " + str(np.std(list_dist)))

Mean(Hamming Loss): 0.22537502895529304
Std.(Hamming Loss): 0.017536634576492193
 
Mean(Hamming Score): 0.7746249710447068
Std.(Hamming Score): 0.017536634576492193
 
Mean(Hamming Distance): 0.6761250868658789
Std.(Hamming Distance): 0.05260990372947659


<h3>ISLR 12.6.2</h3>

![title](hw5.jpeg)